In [1]:
from core import *

sys.path.append("/Users/sakthi.n/Documents/Opensource/code-as-data/")

fdep_path = "/Users/sakthi.n/Documents/Work/codegen/tmp"
G = get_graph(fdep_path)

Found 65 function files to process
Progress: 3/65 files processed (90.6 files/sec, ~0.7s remaining)
Progress: 6/65 files processed (42.7 files/sec, ~1.4s remaining)
Progress: 9/65 files processed (34.8 files/sec, ~1.6s remaining)
Progress: 12/65 files processed (28.6 files/sec, ~1.9s remaining)
Progress: 15/65 files processed (33.0 files/sec, ~1.5s remaining)
Progress: 18/65 files processed (37.6 files/sec, ~1.2s remaining)
Progress: 21/65 files processed (42.3 files/sec, ~1.0s remaining)
Progress: 24/65 files processed (46.0 files/sec, ~0.9s remaining)
Progress: 27/65 files processed (48.2 files/sec, ~0.8s remaining)
Progress: 30/65 files processed (48.1 files/sec, ~0.7s remaining)
Progress: 33/65 files processed (42.1 files/sec, ~0.8s remaining)
Progress: 36/65 files processed (43.6 files/sec, ~0.7s remaining)
Progress: 39/65 files processed (43.8 files/sec, ~0.6s remaining)
Progress: 42/65 files processed (46.7 files/sec, ~0.5s remaining)
Progress: 45/65 files processed (49.4 files/

In [46]:
from typing import Optional, List, Dict
from pydantic import BaseModel


class StateDependencies(BaseModel):
    db: Optional[List[str]]
    env: Optional[List[str]]                          
    redis: Optional[List[str]]                      

class MetaData(BaseModel):
    function_name: str
    behavior_summary: str
    preconditions: List[str]
    postconditions: List[str]
    state_dependencies: Optional[StateDependencies]
    side_effects: List[str]

class ChildCallRelation(BaseModel):
    child: str
    condition: str

class DetailedMetaData(BaseModel):
    function_name: str
    behavior_summary: str
    preconditions: List[str]
    postconditions: List[str]
    state_dependencies: Optional[StateDependencies]
    side_effects: List[str]
    child_call_relationship: List[ChildCallRelation]

In [14]:
root = "App.Routes:generateGetAccountDetails"
g = get_subgraph(G, root)

print("cnt: ",len(g.nodes()))
for node in g.nodes():
    print(node)

cnt:  15
App.Routes:generateGetAccountDetails
Database.GptResponse:createGptResponse
Database.IntegrationTracker:findOneIntegrationTrackerDomainbyId
Flows.GenInstance:getAccountDetailsFunc
Common:toTitleCaseText
Database.GptResponse.Types:fromGtpResponse
App.Utils:getRandomUUID
App.Utils:getCurrTime
Database.GptResponse.Types:toGtpResponse
Database.IntegrationTracker.Types:fromIntegrationTracker
Database.IntegrationTracker:findOneIntegrationTrackerbyId
Common:toTitleCase
Database.GptResponse.Types:parseMetadata
Database.GptResponse.Types:parseGptResult
App.Utils:decodeJSON


In [15]:
g = process_graph(g, response_schema=DetailedMetaData)

✅ Done: Flows.GenInstance:getAccountDetailsFunc
✅ Done: App.Utils:getRandomUUID
✅ Done: App.Utils:getCurrTime
✅ Done: Database.GptResponse.Types:toGtpResponse
✅ Done: Database.IntegrationTracker:findOneIntegrationTrackerbyId
✅ Done: Common:toTitleCase
✅ Done: App.Utils:decodeJSON
✅ Done: Common:toTitleCaseText
✅ Done: Database.IntegrationTracker.Types:fromIntegrationTracker
✅ Done: Database.GptResponse.Types:parseMetadata
✅ Done: Database.GptResponse.Types:parseGptResult
✅ Done: Database.IntegrationTracker:findOneIntegrationTrackerDomainbyId
✅ Done: Database.GptResponse.Types:fromGtpResponse
✅ Done: Database.GptResponse:createGptResponse
✅ Done: App.Routes:generateGetAccountDetails


In [16]:
from pyvis.network import Network

def get_visualize(g):
    net = Network(height="1000px", width="100%", bgcolor="#222222", font_color="white", directed=True)

    # Add all nodes
    for node in g.nodes():
        net.add_node(node, label=node)

    # Add edges with 'condition' as label (if present)
    for source, target in g.edges():
        condition = g[source][target].get("condition", "")
        label = f"{condition}" if condition else ""
        net.add_edge(source, target, label=label)

    net.show("graph.html", notebook=False)

get_visualize(g)

graph.html


In [26]:
node = "Database.GptResponse:createGptResponse"

for c in g.successors(node):
    print(get_metainfo(g, c))

metainfo = get_child_metainfo(G, node)
print(metainfo)
code = g.nodes[node].get("code_string","")
# _, s = process_node(node=node,user_prompt=parent_promptv2,code=code,metainfo=metainfo,response_schema=DetailedMetaData)
# print(s)

FunctionName: Database.GptResponse.Types:fromGtpResponse
Short Summary: Transforms a raw GPT response data structure into a domain-specific `GptResponseDomain` object. It delegates the parsing of the JSON-encoded `gptOutput` and `metadata` fields to child functions and provides a default value for the `typeOfChange` field if it is not present.
Preconditions:
An input record `gptResp` must be provided.
The `_gptOutput` field of the input is expected to be a text value representing JSON, as required by `parseGptResult`.
The `_metadata` field of the input is expected to be a text value representing JSON parsable into a `Metadata` object, as required by `parseMetadata`.
Postconditions:
A `GptResponseDomain` record is returned.
The `gptOutput` field will contain a parsed `Aeson.Value`, defaulting to `Aeson.Null` on parsing failure.
The `metadata` field will contain a parsed `Metadata` object, defaulting to an empty `Metadata` object on parsing failure.
The `typeOfChange` field defaults to `

In [23]:
g.nodes[node]

{'label': 'createGptResponse',
 'node_type': 'Function',
 'function_signature': 'Text\n-> Category\n-> Value\n-> ChangeType\n-> Maybe Int\n-> Flow GptResponseDomain',
 'code_string': 'createGptResponse\n  documentDataId\n  category\n  gptOutput\n  typeOfChange\n  sequenceId\n  = do currTime <- Utils.getCurrTime\n       dbConf <- getEulerDbConf @GptResponseT\n       uuid <- Utils.getRandomUUID\n       let gptResponse\n             = toGtpResponse\n                 $ GptResponseDomain\n                     {id = uuid, documentDataId = documentDataId, category = category,\n                      typeOfChange = typeOfChange, gptOutput = gptOutput,\n                      metadata = Metadata HM.empty, dateCreated = currTime,\n                      lastModified = currTime, sequenceId = fromMaybe 0 sequenceId,\n                      accuracyMetric = Nothing}\n       eResult <- DB.createRowReturning dbConf meshConfig gptResponse\n       case eResult of\n         Right row -> pure $ fromGtpRespon